# LABORATORIO #1 - Deteccion de Phishing
 Autor: Javier Estuardo 19202
 
 Autor: Marco Ramirez   19588


### Parte 1


* 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [3]:
import pandas as pd

df = pd.read_csv('dataset_pishing.csv')
df.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
5710,http://planetfacts.org/ionosphere/,0,0,0,0,0,0,0,0,0,...,1,1,0,355,3298,1074963,0,0,5,legitimate
5557,http://whatsapp-group-invite18-g8hf7jvjw.2waky...,0,0,0,0,0,0,0,0,0,...,1,1,0,175,7129,8386524,1,1,1,phishing
10421,http://newlifeschooloftheology.com/wp-includes...,0,0,0,0,0,0,0,0,0,...,1,0,0,26,-1,0,0,1,0,phishing
1028,http://www.fm-arena.de/,0,1,0,0,0,0,0,0,0,...,0,0,0,0,-1,1512801,0,0,3,legitimate
7390,https://about.me/automationpersonnelsvcs,0,0,0,0,0,0,0,0,0,...,0,0,0,1012,4465,20110,0,0,7,legitimate


* 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset? 

In [4]:
print(df['status'].value_counts())

legitimate    5715
phishing      5715
Name: status, dtype: int64


                El dataset se encuentra balanceado.

* 3. Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 

* 4. Realice  las  modificaciones  necesarias  para  convertir  la  variable  categórica  status  a  una  variable 
binaria. Elimine la columna del dominio. 

* 5. Genere  un  reporte  de  perfil  con  la  librería  pandas_profiling.  Analice  el  reporte  y  determine  las 
columnas que son constantes, o que no tienen una varianza alta con la columna status. Almacene su 
reporte como una página html. 

* 6. En base al análisis del reporte, elimine las características repetidas o irrelevantes para la clasificación 
de un sitio de pishing. Verifique que no posee observaciones repetidas. 

### Parte 2

* 1. Separación de datos

        * Datos de entrenamiento: 55%
 
        * Datos de validación: 15% 
        * Datos de prueba: 30% 
        * Almacene cada dataset como un archivo .csv 


* 2. Utilice dos algoritmos de Machine Learning para entrenar el modelo. Muestre y explique los valores 
obtenidos de las siguientes métricas para los datos de validación y pruebas, para cada modelo, en 
base al contexto del problema (detección de Pishing). 

        * Modelo 1

        * Resultados de modelo 1

        * Modelo 2


        * Resultados del modelo 2

### Discusion

* 1. ¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?  
* 2. ¿Cuál es el impacto de clasificar un sitio de Pishing como legítimo? 
* 3. En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares de clasificación de pishing? 
* 4. ¿Qué modelo es mejor para la clasificación de Pishing? Justifique 
* 5. En base a las métricas obtenidas ¿es necesaria la intervención de una persona humana para tomar la decisión final? 